In [57]:
import pandas as pd
import numpy as np
import ast
import re
import statistics
from sklearn.metrics import roc_auc_score

In [58]:
test_padchest = pd.read_csv('../Data/Data_splits/pathology_detection-test.csv', index_col=0)

In [59]:
preds_bbox_both_mask_inside_df = pd.read_csv("../Predictions/PD_preds_bbox_both_mask_inside.csv")

In [60]:
preds_bbox_both_mask_outside_df = pd.read_csv("../Predictions/PD_preds_bbox_both_mask_outside.csv")

In [61]:
preds_bbox_mask_inside_df = pd.read_csv("../Predictions/PD_preds_bbox_mask_inside.csv")

In [62]:
preds_bbox_mask_outside_df = pd.read_csv("../Predictions/PD_preds_bbox_mask_outside.csv")

In [63]:
preds_dilated_mask_1_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_1_inside.csv")

In [64]:
preds_dilated_mask_1_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_1_outside.csv")

In [65]:
preds_dilated_mask_2_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_2_inside.csv")

In [66]:
preds_dilated_mask_2_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_2_outside.csv")

In [67]:
preds_dilated_mask_3_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_3_inside.csv")

In [68]:
preds_dilated_mask_3_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_3_outside.csv")

In [69]:
preds_dilated_mask_4_inside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_4_inside.csv")

In [70]:
preds_dilated_mask_4_outside_df = pd.read_csv("../Predictions/PD_preds_dilated_mask_4_outside.csv")

In [71]:
preds_original_mask_inside_df = pd.read_csv("../Predictions/PD_preds_original_mask_inside.csv")

In [72]:
preds_original_mask_outside_df = pd.read_csv("../Predictions/PD_preds_original_mask_outside.csv")

In [73]:
preds_bbox_both_mask_inside_df

,Unnamed: 0,Model_name,Val_data,Preds_model1,Preds_model2,Preds_model3
0,0,"PD, Multiclass, Densenet, Imagenet, Fine-tuned",PadChest test set,[[1.30609833e-02 4.78702970e-03 4.23425436e-03...,[[4.55487594e-02 1.59274554e-03 1.64995845e-02...,[[1.04184067e-02 4.47855424e-03 5.36778290e-03...


In [74]:
# Function for reading the predictions, which are strings, as numpy arrays
def str2array(s):
    # Remove space after [
    s=re.sub('\[ +', '[', s.strip())
    # Replace commas and spaces
    s=re.sub('[,\s]+', ', ', s)
    return np.array(ast.literal_eval(s))


# Function to arrange preds nicely in a df
def get_preds_PD(orig_pred_df, true_labels_df, print_auc=True):
    
    pathologies = ['Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    all_preds = []
    
    for row_number in range(len(orig_pred_df)):
        for p_idx, p in enumerate(pathologies):
            preds = [[str2array(i["Preds_model1"]) for idx, i in orig_pred_df.iterrows()][row_number][:,p_idx],
                     [str2array(i["Preds_model2"]) for idx, i in orig_pred_df.iterrows()][row_number][:,p_idx],
                     [str2array(i["Preds_model3"]) for idx, i in orig_pred_df.iterrows()][row_number][:,p_idx]]
            all_preds.append(preds)
            
    # Constructing a df with the preds and 'true' labels
    preds_df = pd.DataFrame(list(zip(list(true_labels_df['ImagePath']),
                                     list(true_labels_df['Effusion']),
                                     list(true_labels_df['Pneumothorax']),
                                     list(true_labels_df['Atelectasis']),
                                     list(true_labels_df['Cardiomegaly']),
                                     list(true_labels_df['Pneumonia']),
                                     list(all_preds[0][0]),
                                     list(all_preds[0][1]),
                                     list(all_preds[0][2]),
                                     list(all_preds[1][0]),
                                     list(all_preds[1][1]),
                                     list(all_preds[1][2]),
                                     list(all_preds[2][0]),
                                     list(all_preds[2][1]),
                                     list(all_preds[2][2]),
                                     list(all_preds[3][0]),
                                     list(all_preds[3][1]),
                                     list(all_preds[3][2]),
                                     list(all_preds[4][0]),
                                     list(all_preds[4][1]),
                                     list(all_preds[4][2]))),
                            columns = ['ImagePath', 'Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly',
                                       'Pneumonia', 'preds_Effusion_model1', 'preds_Effusion_model2', 'preds_Effusion_model3',
                                       'preds_Pneumothorax_model1', 'preds_Pneumothorax_model2', 'preds_Pneumothorax_model3',
                                       'preds_Atelectasis_model1', 'preds_Atelectasis_model2', 'preds_Atelectasis_model3',
                                       'preds_Cardiomegaly_model1', 'preds_Cardiomegaly_model2', 'preds_Cardiomegaly_model3',
                                       'preds_Pneumonia_model1', 'preds_Pneumonia_model2', 'preds_Pneumonia_model3'])


    # Computing the auc for each pathology separately
    for p in pathologies:
        if print_auc:
            print(p)
        auc_list = []
        for i in range(3):
            #print(i+1)
            auc = roc_auc_score(preds_df[p], preds_df['preds_' + str(p) + '_model' + str(i+1)])
            auc_list.append(auc)
        #print(auc_list)
        
        if print_auc:
            print("Average auc:", round(sum(auc_list)/len(auc_list)*100, 1))
        
            print()

    return preds_df

In [75]:
def create_auc_table(dataframes, names):
    pathologies = ['Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    columns = ["name",'Effusion', 'Pneumothorax', 'Atelectasis', 'Cardiomegaly', 'Pneumonia']
    out_dict =  {pathology:list() for pathology in columns}

    for idx in range(len(dataframes)):
        out_dict["name"].append(names[idx])

        for p in pathologies:
            auc = roc_auc_score(dataframes[idx][p], dataframes[idx]['preds_' + str(p) + '_model1'])
            out_dict[p].append(auc)

    return pd.DataFrame(out_dict)

In [76]:
pds_bbox_both_mask_inside_df = get_preds_PD(preds_bbox_both_mask_inside_df, test_padchest)

Effusion
Average auc: 47.5

Pneumothorax
Average auc: 47.5

Atelectasis
Average auc: 46.3

Cardiomegaly
Average auc: 48.9

Pneumonia
Average auc: 49.1



In [77]:
pds_bbox_both_mask_outside_df = get_preds_PD(preds_bbox_both_mask_outside_df, test_padchest) 

Effusion
Average auc: 50.9

Pneumothorax
Average auc: 55.9

Atelectasis
Average auc: 49.1

Cardiomegaly
Average auc: 49.4

Pneumonia
Average auc: 48.3



In [78]:
pds_bbox_mask_inside_df = get_preds_PD(preds_bbox_mask_inside_df, test_padchest)

Effusion
Average auc: 47.1

Pneumothorax
Average auc: 56.3

Atelectasis
Average auc: 44.3

Cardiomegaly
Average auc: 49.3

Pneumonia
Average auc: 48.0



In [79]:
pds_bbox_mask_outside_df = get_preds_PD(preds_bbox_mask_outside_df, test_padchest)

Effusion
Average auc: 50.9

Pneumothorax
Average auc: 52.1

Atelectasis
Average auc: 51.2

Cardiomegaly
Average auc: 49.3

Pneumonia
Average auc: 47.5



In [80]:
pds_dilated_mask_1_inside_df = get_preds_PD(preds_dilated_mask_1_inside_df, test_padchest)

Effusion
Average auc: 49.1

Pneumothorax
Average auc: 43.8

Atelectasis
Average auc: 46.1

Cardiomegaly
Average auc: 49.4

Pneumonia
Average auc: 49.3



In [81]:
pds_dilated_mask_1_outside_df = get_preds_PD(preds_dilated_mask_1_outside_df, test_padchest)

Effusion
Average auc: 49.6

Pneumothorax
Average auc: 53.5

Atelectasis
Average auc: 47.5

Cardiomegaly
Average auc: 49.4

Pneumonia
Average auc: 49.0



In [82]:
pds_dilated_mask_2_inside_df = get_preds_PD(preds_dilated_mask_2_inside_df, test_padchest)

Effusion
Average auc: 49.1

Pneumothorax
Average auc: 47.7

Atelectasis
Average auc: 47.0

Cardiomegaly
Average auc: 49.6

Pneumonia
Average auc: 49.4



In [83]:
pds_dilated_mask_2_outside_df = get_preds_PD(preds_dilated_mask_2_outside_df, test_padchest)

Effusion
Average auc: 49.7

Pneumothorax
Average auc: 54.4

Atelectasis
Average auc: 47.8

Cardiomegaly
Average auc: 49.2

Pneumonia
Average auc: 49.1



In [84]:
pds_dilated_mask_3_inside_df = get_preds_PD(preds_dilated_mask_3_inside_df, test_padchest)

Effusion
Average auc: 49.0

Pneumothorax
Average auc: 51.4

Atelectasis
Average auc: 47.9

Cardiomegaly
Average auc: 49.6

Pneumonia
Average auc: 49.4



In [85]:
pds_dilated_mask_3_outside_df = get_preds_PD(preds_dilated_mask_3_outside_df, test_padchest)

Effusion
Average auc: 49.4

Pneumothorax
Average auc: 52.3

Atelectasis
Average auc: 47.0

Cardiomegaly
Average auc: 49.3

Pneumonia
Average auc: 48.2



In [86]:
pds_dilated_mask_4_inside_df = get_preds_PD(preds_dilated_mask_4_inside_df, test_padchest)

Effusion
Average auc: 48.9

Pneumothorax
Average auc: 52.6

Atelectasis
Average auc: 47.5

Cardiomegaly
Average auc: 49.7

Pneumonia
Average auc: 48.6



In [87]:
pds_dilated_mask_4_outside_df = get_preds_PD(preds_dilated_mask_4_outside_df, test_padchest)

Effusion
Average auc: 49.7

Pneumothorax
Average auc: 51.5

Atelectasis
Average auc: 47.6

Cardiomegaly
Average auc: 49.4

Pneumonia
Average auc: 47.4



In [88]:
pds_original_mask_inside_df = get_preds_PD(preds_original_mask_inside_df, test_padchest)

ValueError: malformed node or string: <_ast.Name object at 0x7f8416ee7590>

In [ ]:
pds_original_mask_outside_df = get_preds_PD(preds_original_mask_outside_df, test_padchest)

ValueError: malformed node or string: <_ast.Name object at 0x7f8413cea590>